In [1]:
from transformers import DistilBertTokenizer, DistilBertModel
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained("distilbert-base-uncased")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [2]:
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')
encoded_input

{'input_ids': tensor([[ 101, 5672, 2033, 2011, 2151, 3793, 2017, 1005, 1040, 2066, 1012,  102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [3]:
output = model(**encoded_input)
output

BaseModelOutput(last_hidden_state=tensor([[[ 4.4081e-04, -2.6241e-01, -1.0192e-01,  ..., -6.2764e-02,
           2.7584e-01,  3.7014e-01],
         [ 7.2233e-01,  1.6449e-01,  4.0025e-01,  ...,  1.9161e-01,
           4.0458e-01, -5.8094e-02],
         [ 2.8198e-01, -1.7430e-01,  3.9076e-02,  ...,  2.7681e-02,
           1.1886e-01,  9.1439e-01],
         ...,
         [ 6.8016e-01,  7.9712e-02,  8.3603e-01,  ..., -4.8959e-01,
          -2.5017e-01, -2.3518e-01],
         [ 3.8105e-02, -8.1751e-01, -3.4076e-01,  ...,  4.4815e-01,
           9.6725e-02, -2.0311e-01],
         [ 3.5750e-01,  1.9968e-01,  1.7437e-01,  ...,  1.5028e-01,
          -2.3665e-01,  5.4391e-02]]], grad_fn=<NativeLayerNormBackward0>), hidden_states=None, attentions=None)

In [4]:
output.last_hidden_state.shape #12 tokens -> 12 embeddings with each embeddings of shape 768

torch.Size([1, 12, 768])

In [ ]:
from transformers import pipeline
unmasker = pipeline('fill-mask', model='distilbert-base-uncased')
unmasker("Hello I'm a [MASK] model.")

Device set to use cpu


[{'score': 0.05292855575680733,
  'token': 2535,
  'token_str': 'role',
  'sequence': "hello i ' m a role model."},
 {'score': 0.0396859310567379,
  'token': 4827,
  'token_str': 'fashion',
  'sequence': "hello i ' m a fashion model."},
 {'score': 0.03474373370409012,
  'token': 2449,
  'token_str': 'business',
  'sequence': "hello i ' m a business model."},
 {'score': 0.03462300822138786,
  'token': 2944,
  'token_str': 'model',
  'sequence': "hello i ' m a model model."},
 {'score': 0.018145214766263962,
  'token': 11643,
  'token_str': 'modeling',
  'sequence': "hello i ' m a modeling model."}]

In [5]:
import pandas as pd
df = pd.read_csv('https://github.com/SkyThonk/Movie-Reviews-Sentiment-Analysis/raw/refs/heads/master/dataset/IMDB%20Dataset.csv')
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [6]:
df.shape

(50000, 2)

In [7]:
df.isna().mean() * 100

,0
review,0.0
sentiment,0.0


In [8]:
df['sentiment'].value_counts()

,count
sentiment,
positive,25000
negative,25000


In [9]:
df_positive = df[df['sentiment'] == 'positive'].sample(n=100, random_state=42)
df_negative = df[df['sentiment'] == 'negative'].sample(n=100, random_state=42)

df_sampled = pd.concat([df_positive, df_negative])
df_sampled['sentiment'].value_counts()

,count
sentiment,
positive,100
negative,100


In [10]:
df_sampled.shape

(200, 2)

In [11]:
import torch
from tqdm.auto import tqdm

tqdm.pandas()
model.eval()

def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze().numpy()
    return cls_embedding

# Apply to your dataframe
df_sampled['embedding'] = df_sampled['review'].progress_apply(get_embedding)

  0%|          | 0/200 [00:00<?, ?it/s]

In [12]:
df_sampled.head()

,review,sentiment,embedding
13886,I don't know how or why this film has a meager...,positive,"[0.033604186, -0.260382, -0.02589136, -0.11841..."
48027,For a long time it seemed like all the good Ca...,positive,"[-0.033207715, -0.1771536, -0.11581796, 0.0154..."
19536,Terry Gilliam's and David Peoples' teamed up t...,positive,"[0.09208056, -0.20113, -0.14131731, -0.0528561..."
27232,What is there to say about an anti-establishme...,positive,"[0.008106758, 0.022867111, -0.4147332, -0.1605..."
28001,This movie was made only 48 years after the en...,positive,"[-0.019682296, -0.1202757, -0.20506854, 0.1784..."


In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import pandas as pd
import numpy as np

In [14]:
X = np.stack(df_sampled['embedding'].values)  # shape: (num_samples, 768)
y = df_sampled['sentiment'].map({'positive': 1, 'negative': 0}).values  # binary labels

In [15]:
X

array([[ 0.03360419, -0.260382  , -0.02589136, ...,  0.08878371,
         0.48237297,  0.38736096],
       [-0.03320771, -0.1771536 , -0.11581796, ..., -0.162242  ,
         0.46921772,  0.38104153],
       [ 0.09208056, -0.20113   , -0.14131731, ..., -0.0434599 ,
         0.64415336,  0.3209259 ],
       ...,
       [ 0.05910736, -0.08410124, -0.19742787, ...,  0.04907332,
         0.6821667 ,  0.4252021 ],
       [ 0.09384809, -0.00695448,  0.02951818, ..., -0.13393278,
         0.5601815 ,  0.37884963],
       [-0.08113395, -0.00743285, -0.31800744, ...,  0.09599333,
         0.6887345 ,  0.52792686]], dtype=float32)

In [16]:
y

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0])

In [17]:
X.shape, y.shape

((200, 768), (200,))

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [19]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((160, 768), (40, 768), (160,), (40,))

In [20]:
clf = LogisticRegression()
clf.fit(X_train, y_train)

LogisticRegression()

In [21]:
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.71      0.89      0.79        19
           1       0.88      0.67      0.76        21

    accuracy                           0.78        40
   macro avg       0.79      0.78      0.77        40
weighted avg       0.80      0.78      0.77        40



In [22]:
accuracy = accuracy_score(y_test, y_pred)
accuracy

0.775